In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_22.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "22_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "22_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "22_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "22_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "22_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "22_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 22_sub_mat_1.csv has been saved.
Submatrix 22_sub_mat_2.csv has been saved.
Submatrix 22_sub_mat_3.csv has been saved.
submatrix 22_cross_matrix_1_2.csv has been saved
submatrix 22_cross_matrix_1_3.csv has been saved
submatrix 22_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0 -0.676449 -0.730779 -0.551948 -0.777248 -0.668098 -0.643695 -0.592568   
21085.0 -0.701374 -0.778831 -0.601988 -0.818344 -0.664061 -0.641854 -0.638515   
21087.0 -0.492987 -0.546698 -0.336114 -0.524725 -0.389556 -0.353054 -0.307902   
21089.0 -0.610274  0.703765 -0.590604  0.735661  0.613185 -0.571791 -0.581713   
21091.0 -0.675851  0.733034 -0.623443 -0.773871 -0.694574 -0.644411 -0.656843   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0  0.878582  0.921677 -0.717077  -0.92429 -0.802796 -0.793155 -0.797941   
38091.0  0.668046  0.621226 -0.585931  0.689227 -0.677088 -0.650255 -0.620907   
38093.0  0.855316  0.871155 -0.738202  -0.90491 -0.780332 -0.769481 -0.747963   
38095.0  0.673241  0.709918 -0.591686  0.766027 -0.673234 -0.634181  -0.63035   
38097.0  0.877783  0.868161 -0.640831  0.877335  -0.68439 -0.731813 -0.666409   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0 -0.637498 -0.637215 -0.643019  ...  0.670524  0.916937  0.871076   
21085.0 -0.609925 -0.625721 -0.680388  ...  0.581273  0.869185  0.817802   
21087.0 -0.339108 -0.375885 -0.428322  ...  0.438348  0.595958  0.566283   
21089.0  -0.60769 -0.605947 -0.602703  ...  0.705095  0.897152  0.887745   
21091.0 -0.641933 -0.625355 -0.686212  ...  0.590068  0.887886  0.806425   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0 -0.871225 -0.775793 -0.901908  ...  0.569142 -0.835735 -0.757349   
38091.0 -0.621209 -0.709623 -0.674617  ...   0.48941 -0.587188  0.555044   
38093.0 -0.822682 -0.796351 -0.874063  ...  0.644323 -0.771938 -0.651465   
38095.0 -0.720718  0.597083 -0.684242  ...   0.46168 -0.692837  0.642056   
38097.0 -0.745991 -0.806498 -0.798345  ...  0.615649  0.774288  0.755379   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0  0.902351  0.807911  0.879144  0.664311  0.864801  0.688571  0.717956  
21085.0  0.863601  0.812524  0.864706 -0.690051  0.831436  0.754178  0.744847  
21087.0  0.583785  0.564887  0.592821 -0.424385  0.556942  0.526133  0.491022  
21089.0   0.86661  0.805121  0.808388  0.753135  0.832644  0.687529  0.724981  
21091.0  0.865652  0.809029  0.856133  0.710606  0.861132  0.737616   0.71693  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0 -0.863535  0.856129 -0.848021 -0.881225 -0.863257  0.778906  0.790805  
38091.0 -0.655521 -0.617998  -0.62031 -0.644638 -0.610567  0.545549  0.592556  
38093.0 -0.795404 -0.843657 -0.780035 -0.773291 -0.832406  0.804038  0.760092  
38095.0 -0.692581  0.740895  0.676681 -0.641205  0.675814  0.644646  0.650584  
38097.0  0.789205  0.837652  0.841981  -0.71386 -0.805035  0.826202  0.819086  

[1000 rows x 1104 columns]